# **Model training Random Forest**

### Package requirements

In [ ]:
!pip install focal-loss

In [ ]:
!pip install plot_keras_history

Load training set

In [ ]:
!unzip data_fly.zip

In [5]:
from Load import*
from Train import*
from Utils import*
from Data_augmentation import*
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **Loading data**

In [6]:
X, Y = load_training_data()
train, test, val = train_te_val_split(X,Y)

## **Building training / validation / testing data sets**

In [8]:
build_model_RF(train[0], train[1])

In [12]:
predict(test[0],test[1],"Random_Forest")

NameError: name 'F1Score' is not defined

## **Cross-validation**

In [14]:
x_train=train[0]
y_train=train[1]

#feature expansion
x_train_expanded=feature_expansion(x_train)

#time window flattening
x_tr_expanded,y_tr_t=time_window_sample(x_tr_expanded, y_tr, 100)

#reshaping
x_train_reshaped=np.reshape(x_train, (x_train.shape[0]*x_train.shape[1], x_train.shape[2]))
y_train_reshaped=np.reshape(y_train, (y_train.shape[0]*y_train.shape[1], y_train.shape[2]))


### Random search

In [18]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 5, stop = 20, num = 1)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 10)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [19]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 50, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(x_train_reshaped,y_train_reshaped)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   57.0s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  4.5min finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=50,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 14, 18, 23, 27, 32,
                                                      36, 41, 45, 50, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [5]},
                   random_state=42, verbose=2)

In [20]:
rf_random.best_params_

{'n_estimators': 5,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': 10,
 'bootstrap': False}

### Grid search

In [16]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True,False],
    'max_depth': [10, 14, 18, 23, 27, 32,36, 41, 45, 50, None],
    'max_features': ['auto', 'sqrt'],
    'min_samples_leaf': [1, 2, 4],
    'min_samples_split': [2, 5, 10],
    'n_estimators': [5, 10, 11, 12]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)

In [17]:
# Fit the grid search to the data
grid_search.fit(x_train_reshaped, y_train_reshaped)
grid_search.best_params_

Fitting 3 folds for each of 1584 candidates, totalling 4752 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   46.6s


KeyboardInterrupt: 